In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from utils.dataset import DataSet, LABELS
from models.own import OwnSingleModel
from keras.utils import to_categorical
from sklearn.metrics import f1_score, accuracy_score
from keras.utils import plot_model

from env import *

%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs/simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

MODEL_FILE = os.path.join(SAVED_MODELS_PATH, 'own/single.h5')
RESULT_CSV = os.path.join(RESULTS_PATH, 'own_single.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

seq = 'word_seq'
train_x = np.array(list(train_with_seq[seq]))
val_x = np.array(list(val_with_seq[seq]))
test_x = np.array(list(test_with_seq[seq]))

y_cols = LABELS
val_ys = []
train_outputs = []
val_outputs = []
for col in y_cols:
    train_y = train_with_seq[col] + 2
    val_y = val_with_seq[col] + 2
    val_ys.append(val_y)
    train_y_onehot = to_categorical(train_y)
    val_y_onehot = to_categorical(val_y)
    train_outputs.append(train_y_onehot)
    val_outputs.append(val_y_onehot)

In [6]:
model = OwnSingleModel(max_len = train_x.shape[1])
# model._model.summary()
plot_model(model._model)

In [7]:
model.fit(train_x, train_outputs, 
          validation_data = (val_x, val_outputs),
          model_file = MODEL_FILE
         )

Train on 105000 samples, validate on 15000 samples
Epoch 1/300
105000/105000 [==============================] - 35s 335us/step - loss: 13.6784 - dense_1_loss: 0.3655 - dense_2_loss: 0.4615 - dense_3_loss: 0.5215 - dense_4_loss: 0.4988 - dense_5_loss: 0.8156 - dense_6_loss: 0.2465 - dense_7_loss: 0.4557 - dense_8_loss: 1.0615 - dense_9_loss: 0.6446 - dense_10_loss: 0.8465 - dense_11_loss: 0.7142 - dense_12_loss: 0.6451 - dense_13_loss: 0.8388 - dense_14_loss: 0.7336 - dense_15_loss: 1.0293 - dense_16_loss: 0.8897 - dense_17_loss: 0.7878 - dense_18_loss: 0.5572 - dense_19_loss: 0.7765 - dense_20_loss: 0.7883 - val_loss: 11.1526 - val_dense_1_loss: 0.2623 - val_dense_2_loss: 0.3539 - val_dense_3_loss: 0.4021 - val_dense_4_loss: 0.4723 - val_dense_5_loss: 0.6143 - val_dense_6_loss: 0.1465 - val_dense_7_loss: 0.3627 - val_dense_8_loss: 0.8644 - val_dense_9_loss: 0.4842 - val_dense_10_loss: 0.5884 - val_dense_11_loss: 0.5802 - val_dense_12_loss: 0.5674 - val_dense_13_loss: 0.7503 - val_dense

105000/105000 [==============================] - 33s 313us/step - loss: 5.8740 - dense_1_loss: 0.1288 - dense_2_loss: 0.1587 - dense_3_loss: 0.1920 - dense_4_loss: 0.2545 - dense_5_loss: 0.2934 - dense_6_loss: 0.0844 - dense_7_loss: 0.1816 - dense_8_loss: 0.4373 - dense_9_loss: 0.2783 - dense_10_loss: 0.3567 - dense_11_loss: 0.3059 - dense_12_loss: 0.2792 - dense_13_loss: 0.4466 - dense_14_loss: 0.2763 - dense_15_loss: 0.4444 - dense_16_loss: 0.4408 - dense_17_loss: 0.4305 - dense_18_loss: 0.2320 - dense_19_loss: 0.3687 - dense_20_loss: 0.2839 - val_loss: 11.2011 - val_dense_1_loss: 0.2916 - val_dense_2_loss: 0.4395 - val_dense_3_loss: 0.4421 - val_dense_4_loss: 0.3735 - val_dense_5_loss: 0.6034 - val_dense_6_loss: 0.1357 - val_dense_7_loss: 0.3464 - val_dense_8_loss: 0.8096 - val_dense_9_loss: 0.4339 - val_dense_10_loss: 0.5683 - val_dense_11_loss: 0.6241 - val_dense_12_loss: 0.5974 - val_dense_13_loss: 0.7608 - val_dense_14_loss: 0.6094 - val_dense_15_loss: 0.8372 - val_dense_16_loss

In [8]:
model._model.load_weights(MODEL_FILE)
val_probs = model.predict(val_x)
test_probs = model.predict(test_x)

val_preds = list(map(lambda x: np.argmax(x, axis = -1), val_probs))
# val_preds = np.argmax(val_probs, axis = 1)
# test_preds = list(map(lambda x: np.argmax(x, axis = -1), test_probs))

In [10]:
f1s = 0
for i, (vy, vp) in enumerate(zip(val_ys, val_preds)):
    f1 = f1_score(vy, vp, average='macro')
    print("The %sth f1: %s" % (i, f1))
    f1s += f1
    
print("The average f1 of val is %s" % (f1s / len(y_cols)))

The 0th f1: 0.4596354033490769
The 1th f1: 0.3929970327642951
The 2th f1: 0.5295040983037937
The 3th f1: 0.4305639940330617
The 4th f1: 0.6912658532749753
The 5th f1: 0.4937591469268501
The 6th f1: 0.5591111037496518
The 7th f1: 0.6408409780561068
The 8th f1: 0.4915788789772023
The 9th f1: 0.5435962254233646
The 10th f1: 0.5277953515975469
The 11th f1: 0.5414153841463157
The 12th f1: 0.49896178686251547
The 13th f1: 0.550762413602188
The 14th f1: 0.5663567536545737
The 15th f1: 0.5218138169250428
The 16th f1: 0.40697435509378554
The 17th f1: 0.48924305539901836
The 18th f1: 0.48875205269084987
The 19th f1: 0.5445940121789128
The average f1 of val is 0.5184760848504564


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
